# Exemplo de um código em Python para extrair dados de emails do Microsoft 365

primeiro passo para acessar a API é gerar um Bearer Token usando o Client ID, Tenant ID e Client Secret:

In [2]:
import requests

TENANT_ID = "SEU_TENANT_ID"
CLIENT_ID = "SEU_CLIENT_ID"
CLIENT_SECRET = "SEU_CLIENT_SECRET"

Url para obter o token

In [3]:
token_url = f"https://login.microsoftonline.com/{TENANT_ID}/oauth2/v2.0/token"

Parâmetros para a requisição

In [5]:
data = {
    "grant_type": "client_credentials",
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "scope": "https://graph.microsoft.com/.default"
}

Requisição para obter o token

In [6]:
response = requests.post(token_url, data=data)
access_token = response.json().get("access_token")

if access_token:
    print("Token gerado com sucesso!")
else:
    print("Erro ao gerar o token:", response.json())

Erro ao gerar o token: {'error': 'invalid_request', 'error_description': "AADSTS900023: Specified tenant identifier 'seu_tenant_id' is neither a valid DNS name, nor a valid external domain. Trace ID: bb610e3b-cac0-4727-80b1-4631a0ce1a00 Correlation ID: 55fbbc12-9340-4d06-aa4b-a9497bc5d82e Timestamp: 2025-02-06 19:40:31Z", 'error_codes': [900023], 'timestamp': '2025-02-06 19:40:31Z', 'trace_id': 'bb610e3b-cac0-4727-80b1-4631a0ce1a00', 'correlation_id': '55fbbc12-9340-4d06-aa4b-a9497bc5d82e', 'error_uri': 'https://login.microsoftonline.com/error?code=900023'}


### Buscar os emails de um usuário

Agora que temos o access_token, podemos usá-lo para acessar os e-mails de um usuário específico:

In [7]:
def get_user_emails(user_email):
    """Busca os últimos 10 e-mails de um usuário no Microsoft 365"""
    url = f"https://graph.microsoft.com/v1.0/users/{user_email}/messages"
    headers = {"Authorization": f"Bearer {access_token}"}

    response = requests.get(url, headers=headers)
    emails = response.json()

    for email in emails.get("value", []):
        sender = email["from"]["emailAddress"]["address"]
        recipients = [to["emailAddress"]["address"] for to in email.get("toRecipients", [])]
        date = email["receivedDateTime"]

        print(f"De: {sender}")
        print(f"Para: {recipients}")
        print(f"Data: {date}")
        print("-" * 50)

# Exemplo de uso
get_user_emails("usuario@empresa.com")

### Iterar por todos os usuários da Empresa

Para obter os e-mails de todos os usuários primeiro precisamos obter a lista de usuários e depois iterar por cada um

In [8]:
def get_all_users():
    """Lista todos os usuários do Microsoft 365"""
    url = "https://graph.microsoft.com/v1.0/users"
    headers = {"Authorization": f"Bearer {access_token}"}

    response = requests.get(url, headers=headers)
    users = response.json()

    return [user["mail"] for user in users.get("value", []) if user.get("mail")]

busca usuários e extrai emails

In [9]:
all_users = get_all_users()
for user in all_users:
    get_user_emails(user)